In [ ]:
!pip install ollama
!pip install langchain chromadb gradio 
!pip install -U langchain-community
!pip install pymupdf

In [1]:
# Import necessary packages
import ollama  
import gradio as gr  

# Document processing and retrieval  
from langchain_community.document_loaders import PyMuPDFLoader  
from langchain.text_splitter import RecursiveCharacterTextSplitter  
from langchain.vectorstores import Chroma  

# Embedding generation  
#from langchain_community.embeddings import OllamaEmbeddings
from langchain_ollama import OllamaEmbeddings


import re  

In [2]:
# Call the Ollama model to generate a response  
response = ollama.chat(
    model="deepseek-r1:8b",  
    messages=[
        {"role": "user", "content": "Explain Embeddings and Vectors in AI."},  # User's input query
    ],
)

# Print the chatbot's response
print(response["message"]["content"])  

<think>
Okay, so I need to explain embeddings and vectors in AI. Hmm, where do I start? Well, I know that in machine learning, data is often represented as features or nodes in a neural network. But what exactly are embeddings?

I think embeddings are like a way to convert text into numerical representations. Wait, but isn't that similar to word embeddings? Yeah, maybe they're used in areas like natural language processing. So instead of just having a one-hot vector for each word, which can be inefficient, embeddings create continuous vectors where each word's vector captures its meaning.

So, for example, if I have the words "man", "woman", and "apple", their embeddings would reflect that "woman" is related to "man" because they share similar components. And "apple" might have different components altogether, maybe not as related to the others.

Vectors, on the other hand, are like these continuous representations. They can be of any dimension but usually lower than the input space. T

In [3]:
# Define the function that processes the PDF
def process_pdf(pdf_bytes):
    if pdf_bytes is None:
        return None, None, None

    loader = PyMuPDFLoader(pdf_bytes) 
    data = loader.load()

    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
    chunks = text_splitter.split_documents(data)

    embeddings = OllamaEmbeddings(model="nomic-embed-text:latest")

    vectorstore=Chroma.from_documents(documents=chunks, embedding=embeddings, persist_directory="./chroma_db")  # Example directory
    retriever = vectorstore.as_retriever()
    """
    The function returns 3 objects
        text_splitter → (Used to split new text in the same way as before)
        vectorstore → (Holds the processed document chunks)
        retriever → (Used to fetch relevant document chunks when answering questions)
    """
    
    return text_splitter, vectorstore, retriever

In [4]:
def combine_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [5]:
def ollama_llm(question, context):

    formatted_prompt = f"Question: {question}\n\nContext: {context}"

    response = ollama.chat(
        model="deepseek-r1:8b",  # Specifies the AI model to use
        messages=[{'role': 'user', 'content': formatted_prompt}]  # Formats the user input
    )
    response_content = response['message']['content']

    final_answer = re.sub(r'<think>.*?</think>', # We're searching for think tags
                          '', # We'll replace them with empty spaces
                          response_content, # In response_content
                          flags=re.DOTALL).strip() # (dot) should match newlines (\n) as well.
    # Return the final cleaned response
    return final_answer

In [6]:
# Define rag_chain function for Retrieval Augmented Generation
def rag_chain(question, text_splitter, vectorstore, retriever):
    """
    This function takes as input:
        - The question we want to ask the model
        - The text_splitter object to split the PDF and read into chunks
        - The vectorstore for retrieving embeddings 
        - The retriever objects which retrieves data from the vectorstore
    """
    retrieved_docs = retriever.invoke(question) 
    formatted_content = combine_docs(retrieved_docs) 
    return ollama_llm(question, formatted_content)  

In [ ]:
def ask_question(pdf_bytes, question): 
    text_splitter, vectorstore, retriever = process_pdf(pdf_bytes) # Process the PDF
    if text_splitter is None:
        return None  # No PDF uploaded    
    result = rag_chain(question, text_splitter, vectorstore, retriever) # Return the results with RAG
    #return {result}
    return result

In [8]:
# Define a Gradio interface
interface = gr.Interface(
    fn=ask_question,  
    inputs=[
        gr.File(label="Upload PDF (optional)"),  
        gr.Textbox(label="Ask a question")  
    ],
    #outputs="text",  
    outputs=gr.Markdown(),
    title="Ask questions about your PDF",  
    description="Use Ollama model to answer your questions about the uploaded PDF document.",  
)

# Launch the Gradio interface to start the web-based app
interface.launch()

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "c:\Users\Muthu\.conda\envs\langchain_py311\Lib\site-packages\gradio\queueing.py", line 625, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Muthu\.conda\envs\langchain_py311\Lib\site-packages\gradio\route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Muthu\.conda\envs\langchain_py311\Lib\site-packages\gradio\blocks.py", line 2106, in process_api
    data = await self.postprocess_data(block_fn, result["prediction"], state)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Muthu\.conda\envs\langchain_py311\Lib\site-packages\gradio\blocks.py", line 1912, in postprocess_data
    prediction_value = block.postprocess(prediction_value)
                       ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\